##  1. Imports

In [9]:
import requests
import numpy as np
import json
import pandas as pd
import sys
import collections
import scipy
import matplotlib.pyplot as plt
from PIL import Image
import ibmseti
import os,sys
sys.path.append('/home/cs341seti/cs341-ibm-seti/')
import numpy as np
import pandas as pd
import ibmseti
import collections
import scipy.io
import matplotlib.pyplot as plt
import commonutils as cu
import PIL
from PIL import Image
import model_specs

## 2. Gathered the File list store in a data frame

In [10]:
# Downloading and saving all files to local
r = requests.get('https://dal.objectstorage.open.softlayer.com/v1/AUTH_cdbef52bdf7a449c96936e1071f0a46b/simsignals_files/public_list_headers_3april_2017.txt')
fileList = r.text.split('\n')[:-1]
fileListJSON = [json.loads(entry) for entry in fileList]
# Create a data frame with all the file info
fileListTuples = [(str(j['file_name']),str(j['signal_classification']),str(j['uuid'])) for j in fileListJSON]
fileListDF = pd.DataFrame.from_records(fileListTuples,columns=["file_name","signal_classification","uuid"])
# Creating a file index. Will use this for filenames for easier reference
fileListDF['file_index'] = fileListDF.index
fileListDF['file_index'] = fileListDF.file_index.apply(lambda x: str.zfill(str(x),6) )
## Assigning numbers to classes
labelMap = {'noise':0,'brightpixel':1,'narrowband':2,
            'narrowbanddrd':3,'squarepulsednarrowband':4,'squiggle':5,'squigglesquarepulsednarrowband':6}
fileListDF["label"] = fileListDF["signal_classification"].apply(lambda x: labelMap[x])
print labelMap
fileListDF.to_csv("fileList.csv",index=False)

{'noise': 0, 'squiggle': 5, 'narrowbanddrd': 3, 'brightpixel': 1, 'narrowband': 2, 'squigglesquarepulsednarrowband': 6, 'squarepulsednarrowband': 4}


## 3a. Functionality to download and save the Welch periodogram as a numpy array

In [6]:
# Function: input - JSON line with url, output - spectrogram array
def downloadSaveSignalData(DFrow,base_url,container,data_path):
    #********************Configuration Parameters Start************************#
    # This is the expected pixel width of the output image, height is determined automatically
    desiredSignalWidth = 512
    assert desiredSignalWidth&31 == 0 , "Desired Image width should be a multiple of 32"
    # How much should should the periodogram frequency be scaled by determines periodogram length
    periodogramFrequencyScaling = 4
    # rotatePSD - accounts for the 90 degree rotation caused by the Welch Periodogram
    rotatePsd = True
    # Normalize PSD
    NormalizePsd = True
    # Gauss Filter PSD
    GaussFilterPsd = True
    # Number of columns eliminated fromthe periodogram to preven the vertical line around the center frequency
    numEliminatedColumns = 2
    #********************Configuration Parameters End**************************#
    # Access file via HTTP method
    fname = DFrow['file_name']
    print "\r{}".format(DFrow["file_index"]),
    r = requests.get('{}/{}/{}'.format(base_url, container, fname))   
    aca = ibmseti.compamp.SimCompamp(r.content)
    originalSignal = aca.get_signal()
    # 2. Get the signal dimensions
    rawSignalLength = originalSignal.shape[0]
    # 3. Finding the closest multiple of 32 for final image width
    desiredSignalWidth = ((desiredSignalWidth)&(~31))
    #  This is also the segment length of the periodoogram(num frequency bins)
    npersegIn = desiredSignalWidth
    # 4. Signal width as determined by the frequency scaling needed for noise rejection
    signalWidth = desiredSignalWidth * periodogramFrequencyScaling+numEliminatedColumns
    # Finding the scaled signal length to accommodate width as a multiple of desiredSignalWidth*periodogramFrequencyScaling and height as a multiple of 32
    scaledSignalLength = signalWidth * ((rawSignalLength/signalWidth)&(~31))
    #print(scaledSignalLength)
    signalHeight = scaledSignalLength/signalWidth
    scaledSignal = originalSignal[0:scaledSignalLength]
    # 5. Reshaping the scaled signal
    reshapedSignal = aca._reshape(scaledSignal,(signalHeight,signalWidth))
    # 6. Computing the periodogoram
    from scipy import signal
    welchPeriodogram = signal.welch(reshapedSignal,fs= 8000, nperseg=npersegIn,return_onesided=False)
    powerSpectralDensity = welchPeriodogram[1]
    # 7. Rotating the periodogoram + gauss filtering + normalizing
    rotatedPSD = np.hstack((powerSpectralDensity[:,256:(desiredSignalWidth - numEliminatedColumns/2)],powerSpectralDensity[:,numEliminatedColumns/2:desiredSignalWidth/2]))
    rotatedPSDGaussFiltered = rotatedPSD
    if(GaussFilterPsd):
        rotatedPSDGaussFiltered = scipy.ndimage.filters.gaussian_filter(rotatedPSD,sigma=1,order=0,mode="nearest")        
    rotatedNormalizedPSDGaussFiltered = rotatedPSDGaussFiltered
    if(NormalizePsd):
        rotatedNormalizedPSDGaussFiltered = rotatedPSDGaussFiltered/np.linalg.norm(rotatedPSDGaussFiltered)
    np.save("{}/{}.npy".format(data_path,DFrow['file_index']),rotatedNormalizedPSDGaussFiltered) 

## 3b. Download all periodogram data, save as a set of numpy arrays (skip if already exists)

In [7]:
base_url = 'https://dal.objectstorage.open.softlayer.com/v1/AUTH_cdbef52bdf7a449c96936e1071f0a46b'
container = 'simsignals'
data_path = "../data/specdataraw"
### Iterate through fileListDF and save the periodogram of the signal data as .npy file
### Use file_index for filename
temp = fileListDF.apply(downloadSaveSignalData,axis=1,args=(base_url,container,data_path))
del temp

008187                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

015984                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

## Converting all files to images (skip if already exists)

In [ ]:
#********************Configuration Parameters Start************************#
numEliminatedColumns = 2
desiredImageWidth = 224
desiredImageHeight = 224
#********************Configuration Parameters End**************************#
specPath = "../data/specdataraw/"
specfiles = [f for f in os.listdir(specPath) if os.path.isfile(os.path.join(specPath, f))]
for i in range(len(specfiles)):
    # Convert and save each file as image
    fname = specfiles[i].split(".")[0]
    print "\r{0}".format(i),
    spec = np.load("{}{}.npy".format(specPath,fname))
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.axis('off')
    ax.frameon = False
    fig.frameon = False
    ax.imshow((spec), aspect = 0.5*float(spec.shape[1]) / spec.shape[0],cmap="viridis",interpolation='nearest')
    # Saving the plot
    savePath = "../data/specdataimages_{}x{}/".format(desiredImageWidth,desiredImageHeight)
    print("{}{}.png".format(savePath,fname))
    plt.savefig("{}{}.png".format(savePath,fname))
    # reopening 
    from PIL import Image
    img = Image.open("{}{}.png".format(savePath,fname))
    width = img.size[0]
    height = img.size[1]
    cropped_img = img.crop((200, 100, width-200, height-100))
    cropped_img = cropped_img.resize((desiredImageWidth,desiredImageWidth), Image.LANCZOS)
    cropped_img.save('{}{}.jpeg'.format(savePath,fname))
    del fig

0 ../data/specdataimages_224x224/015545.png
1 ../data/specdataimages_224x224/006933.png
2 ../data/specdataimages_224x224/007491.png
3 ../data/specdataimages_224x224/000525.png
4 ../data/specdataimages_224x224/012655.png
5 ../data/specdataimages_224x224/001594.png
6 ../data/specdataimages_224x224/014279.png
7 ../data/specdataimages_224x224/009950.png
8 ../data/specdataimages_224x224/012482.png
9 ../data/specdataimages_224x224/007872.png
10 ../data/specdataimages_224x224/008652.png
11 ../data/specdataimages_224x224/003776.png
12 ../data/specdataimages_224x224/013965.png
13 ../data/specdataimages_224x224/011441.png
14 ../data/specdataimages_224x224/015813.png
15 ../data/specdataimages_224x224/010644.png
16 ../data/specdataimages_224x224/005589.png
17 ../data/specdataimages_224x224/007041.png
18 ../data/specdataimages_224x224/005614.png
19 ../data/specdataimages_224x224/009921.png
20 ../data/specdataimages_224x224/013208.png
21 ../data/specdataimages_224x224/007730.png
22 ../data/specdatai

181 ../data/specdataimages_224x224/006488.png
182 ../data/specdataimages_224x224/006464.png
183 ../data/specdataimages_224x224/010802.png
184 ../data/specdataimages_224x224/014865.png
185 ../data/specdataimages_224x224/011101.png
186 ../data/specdataimages_224x224/002469.png
187 ../data/specdataimages_224x224/007976.png
188 ../data/specdataimages_224x224/009191.png
189 ../data/specdataimages_224x224/014733.png
190 ../data/specdataimages_224x224/008939.png
191 ../data/specdataimages_224x224/014965.png
192 ../data/specdataimages_224x224/001300.png
193 ../data/specdataimages_224x224/010730.png
194 ../data/specdataimages_224x224/011496.png
195 ../data/specdataimages_224x224/004477.png
196 ../data/specdataimages_224x224/003729.png
197 ../data/specdataimages_224x224/001897.png
198 ../data/specdataimages_224x224/012162.png
199 ../data/specdataimages_224x224/014607.png
200 ../data/specdataimages_224x224/015726.png
201 ../data/specdataimages_224x224/014911.png
202 ../data/specdataimages_224x224

360 ../data/specdataimages_224x224/010124.png
361 ../data/specdataimages_224x224/005578.png
362 ../data/specdataimages_224x224/004503.png
363 ../data/specdataimages_224x224/014820.png
364 ../data/specdataimages_224x224/013037.png
365 ../data/specdataimages_224x224/004343.png
366 ../data/specdataimages_224x224/014616.png
367 ../data/specdataimages_224x224/006519.png
368 ../data/specdataimages_224x224/006107.png
369 ../data/specdataimages_224x224/011060.png
370 ../data/specdataimages_224x224/007449.png
371 ../data/specdataimages_224x224/000547.png
372 ../data/specdataimages_224x224/013239.png
373 ../data/specdataimages_224x224/002786.png
374 ../data/specdataimages_224x224/003404.png
375 ../data/specdataimages_224x224/012347.png
376 ../data/specdataimages_224x224/010562.png
377 ../data/specdataimages_224x224/004491.png
378 ../data/specdataimages_224x224/012639.png
379 ../data/specdataimages_224x224/000144.png
380 ../data/specdataimages_224x224/011676.png
381 ../data/specdataimages_224x224

539 ../data/specdataimages_224x224/015246.png
540 ../data/specdataimages_224x224/015375.png
541 ../data/specdataimages_224x224/007072.png
542 ../data/specdataimages_224x224/013120.png
543 ../data/specdataimages_224x224/014042.png
544 ../data/specdataimages_224x224/012159.png
545 ../data/specdataimages_224x224/009785.png
546 ../data/specdataimages_224x224/001996.png
547 ../data/specdataimages_224x224/001432.png
548 ../data/specdataimages_224x224/010200.png
549 ../data/specdataimages_224x224/011806.png
550 ../data/specdataimages_224x224/005742.png
551 ../data/specdataimages_224x224/012823.png
552 ../data/specdataimages_224x224/013853.png
553 ../data/specdataimages_224x224/002678.png
554 ../data/specdataimages_224x224/013095.png
555 ../data/specdataimages_224x224/007543.png
556 ../data/specdataimages_224x224/002557.png
557 ../data/specdataimages_224x224/013076.png
558 ../data/specdataimages_224x224/004204.png
559 ../data/specdataimages_224x224/009155.png
560 ../data/specdataimages_224x224

718 ../data/specdataimages_224x224/005639.png
719 ../data/specdataimages_224x224/015385.png
720 ../data/specdataimages_224x224/000375.png
721 ../data/specdataimages_224x224/012687.png
722 ../data/specdataimages_224x224/008449.png
723 ../data/specdataimages_224x224/005467.png
724 ../data/specdataimages_224x224/013440.png
725 ../data/specdataimages_224x224/000094.png
726 ../data/specdataimages_224x224/009486.png
727 ../data/specdataimages_224x224/011826.png
728 ../data/specdataimages_224x224/004465.png
729 ../data/specdataimages_224x224/008553.png
730 ../data/specdataimages_224x224/014826.png
731 ../data/specdataimages_224x224/008704.png
732 ../data/specdataimages_224x224/003165.png
733 ../data/specdataimages_224x224/012622.png
734 ../data/specdataimages_224x224/004901.png
735 ../data/specdataimages_224x224/013563.png
736 ../data/specdataimages_224x224/014247.png
737 ../data/specdataimages_224x224/005158.png
738 ../data/specdataimages_224x224/009623.png
739 ../data/specdataimages_224x224

...

5624 ../data/specdataimages_224x224/010118.png
5625 ../data/specdataimages_224x224/009075.png
5626 ../data/specdataimages_224x224/013912.png
5627 ../data/specdataimages_224x224/000771.png
5628 ../data/specdataimages_224x224/009326.png
5629 ../data/specdataimages_224x224/007038.png
5630 ../data/specdataimages_224x224/014810.png
5631 ../data/specdataimages_224x224/013716.png
5632 ../data/specdataimages_224x224/009210.png
5633 ../data/specdataimages_224x224/009830.png
5634 ../data/specdataimages_224x224/000090.png
5635 ../data/specdataimages_224x224/000165.png
5636 ../data/specdataimages_224x224/003125.png
5637 ../data/specdataimages_224x224/000499.png
5638 ../data/specdataimages_224x224/012130.png
5639 ../data/specdataimages_224x224/003719.png
5640 ../data/specdataimages_224x224/004651.png
5641 ../data/specdataimages_224x224/001466.png
5642 ../data/specdataimages_224x224/000595.png
5643 ../data/specdataimages_224x224/009029.png
5644 ../data/specdataimages_224x224/008471.png
5645 ../data/

5799 ../data/specdataimages_224x224/009699.png
5800 ../data/specdataimages_224x224/008822.png
5801 ../data/specdataimages_224x224/013449.png
5802 ../data/specdataimages_224x224/010957.png
5803 ../data/specdataimages_224x224/001001.png
5804 ../data/specdataimages_224x224/001022.png
5805 ../data/specdataimages_224x224/004976.png
5806 ../data/specdataimages_224x224/000099.png
5807 ../data/specdataimages_224x224/001944.png
5808 ../data/specdataimages_224x224/006861.png
5809 ../data/specdataimages_224x224/009658.png
5810 ../data/specdataimages_224x224/006323.png
5811 ../data/specdataimages_224x224/001284.png
5812 ../data/specdataimages_224x224/013526.png
5813 ../data/specdataimages_224x224/013625.png
5814 ../data/specdataimages_224x224/013373.png
5815 ../data/specdataimages_224x224/013722.png
5816 ../data/specdataimages_224x224/007102.png
5817 ../data/specdataimages_224x224/013339.png
5818 ../data/specdataimages_224x224/013690.png
5819 ../data/specdataimages_224x224/001056.png
5820 ../data/

5974 ../data/specdataimages_224x224/008715.png
5975 ../data/specdataimages_224x224/004553.png
5976 ../data/specdataimages_224x224/009372.png
5977 ../data/specdataimages_224x224/003897.png
5978 ../data/specdataimages_224x224/004980.png
5979 ../data/specdataimages_224x224/001807.png
5980 ../data/specdataimages_224x224/001077.png
5981 ../data/specdataimages_224x224/011551.png
5982 ../data/specdataimages_224x224/001213.png
5983 ../data/specdataimages_224x224/000824.png
5984 ../data/specdataimages_224x224/004499.png
5985 ../data/specdataimages_224x224/006456.png
5986 ../data/specdataimages_224x224/014572.png
5987 ../data/specdataimages_224x224/002669.png
5988 ../data/specdataimages_224x224/005285.png
5989 ../data/specdataimages_224x224/008606.png
5990 ../data/specdataimages_224x224/003548.png
5991 ../data/specdataimages_224x224/011772.png
5992 ../data/specdataimages_224x224/011648.png
5993 ../data/specdataimages_224x224/009169.png
5994 ../data/specdataimages_224x224/013183.png
5995 ../data/

6149 ../data/specdataimages_224x224/005068.png
6150 ../data/specdataimages_224x224/008282.png
6151 ../data/specdataimages_224x224/005841.png
6152 ../data/specdataimages_224x224/001328.png
6153 ../data/specdataimages_224x224/005801.png
6154 ../data/specdataimages_224x224/006868.png
6155 ../data/specdataimages_224x224/009312.png
6156 ../data/specdataimages_224x224/009681.png
6157 ../data/specdataimages_224x224/003139.png
6158 ../data/specdataimages_224x224/014028.png
6159 ../data/specdataimages_224x224/003997.png
6160 ../data/specdataimages_224x224/004552.png
6161 ../data/specdataimages_224x224/011267.png
6162 ../data/specdataimages_224x224/013152.png
6163 ../data/specdataimages_224x224/008821.png
6164 ../data/specdataimages_224x224/008848.png
6165 ../data/specdataimages_224x224/002832.png
6166 ../data/specdataimages_224x224/010965.png
6167 ../data/specdataimages_224x224/011543.png
6168 ../data/specdataimages_224x224/008256.png
6169 ../data/specdataimages_224x224/012041.png
6170 ../data/

6324 ../data/specdataimages_224x224/001343.png
6325 ../data/specdataimages_224x224/013611.png
6326 ../data/specdataimages_224x224/007019.png
6327 ../data/specdataimages_224x224/015024.png
6328 ../data/specdataimages_224x224/004278.png
6329 ../data/specdataimages_224x224/000488.png
6330 ../data/specdataimages_224x224/010165.png
6331 ../data/specdataimages_224x224/014164.png
6332 ../data/specdataimages_224x224/000919.png
6333 ../data/specdataimages_224x224/002109.png
6334 ../data/specdataimages_224x224/014023.png
6335 ../data/specdataimages_224x224/012708.png
6336 ../data/specdataimages_224x224/000523.png
6337 ../data/specdataimages_224x224/001959.png
6338 ../data/specdataimages_224x224/002027.png
6339 ../data/specdataimages_224x224/003956.png
6340 ../data/specdataimages_224x224/005355.png
6341 ../data/specdataimages_224x224/010991.png
6342 ../data/specdataimages_224x224/000781.png
6343 ../data/specdataimages_224x224/015209.png
6344 ../data/specdataimages_224x224/004789.png
6345 ../data/

## Building the VGG model to consume 224x224x3 images

In [5]:
model = model_specs.vgg_fine_tune.build((224,224,3),4,
                                "./savedModels/224x224_1024-256-256_4class_rmsproplr0.0001decay1e-06.hdf5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________